# Processing COMTRADE flow data

This script attempts to process the COMTRADEflow data for energy, coal, oil and gas. The goal is to extract the import, export, and flow data as geospatial information: the tools for which are found in the infra_tools library

In [1]:
import sys, os, importlib

import pandas as pd
import geopandas as gpd
import numpy as np

from shapely.geometry import Point, LineString

sys.path.append('../')

from infrasap import process_flows

In [2]:
input_file_energy = "/home/wb411133/data/Projects/INFRA/FLOWS/UN_comtrade_energy_flows.csv"
input_file_coal = "/home/wb411133/data/Projects/INFRA/FLOWS/Coal Ex-Im_UN Comtreade_June 18.csv"
input_file_oil = "/home/wb411133/data/Projects/INFRA/FLOWS/Oil UNComtrade 0115.csv"
input_file_gas = "/home/wb411133/data/Projects/INFRA/FLOWS/Gas UNComtrade 0115.csv"
out_folder = ""
global_boundaries = "/home/wb411133/data/Projects/INFRA/FLOWS/national_centroids.shp"

inB = gpd.read_file(global_boundaries)
if inB.crs != {'init':'epsg:4326'}:
    inB = inB.to_crs({'init':'epsg:4326'})



In [50]:
xx = pd.read_csv(input_file_oil)

In [57]:
xx.columns

Index(['Classification', 'Year', 'Period', 'Period Desc.', 'Aggregate Level',
       'Is Leaf Code', 'Trade Flow Code', 'Trade Flow', 'Reporter Code',
       'Reporter', 'Reporter ISO', 'Partner Code', 'Partner', 'Partner ISO',
       'Commodity Code', 'Commodity', 'Qty Unit Code', 'Qty Unit', 'Qty',
       'Netweight (kg)', 'Gross weight (kg)', 'Trade Value (US$)',
       'CIF Trade Value (US$)', 'FOB Trade Value (US$)', 'Flag'],
      dtype='object')

In [59]:
xx['Commodity'].value_counts()

PETROLEUM PRODUCTS          106995
RESIDUAL PETROL.PRODUCTS     50483
PETROLEUM OILS, CRUDE        16415
Name: Commodity, dtype: int64

In [61]:
xx['Commodity Code'].value_counts()

334    106995
335     50483
333     16415
Name: Commodity Code, dtype: int64

In [63]:
tempD = xx.loc[xx['Partner ISO'] != 'WLD']
tempD = tempD.loc[tempD['Commodity Code'] == 333]
tempD = tempD.loc[:,['Reporter ISO', 'Partner ISO', 'Year', 'Qty', 'Trade Value (US$)']]
tempD.loc[tempD['Reporter ISO'].isin(['AUS','USA','CAN', 'NZL','JPN'])].groupby(['Reporter ISO', 'Year']).sum() / 1000000

Qty  Trade Value (US$)
Reporter ISO Year                                  
AUS          2010   40702.438529       25141.974810
             2011   39313.926720       33326.342681
             2012   39051.561170       33873.966128
             2013   33391.509260       28515.907613
             2015   27913.778390       11800.878595
             2016   27280.221560        9431.260780
             2017   29882.180730       10698.516850
CAN          2010  169470.545319       73582.693018
             2011  174779.839789       98301.617297
             2012   35617.493050      104592.553471
             2013   31371.920113      105610.917037
             2015  131081.069275       63380.872687
             2016  132231.361509       50376.380128
             2017  144695.989008       67208.698235
JPN          2010  178681.440200      105814.289560
             2011  175010.970696      142101.309769
             2012  179676.342412      153107.428715
             2013  178436.733091      145720.786582
             2015   64786.157758       45005.802948
             2016  164423.131785       50767.543409
             2017  157989.308600       63750.784437
NZL          2010    7158.279000        4323.707542
             2011    7280.689201        7729.662673
             2012    6961.020000        6096.155235
             2013    6526.736141        5575.930750
             2015    6341.475249        2741.274683
             2016    6461.355632        2190.106862
             2017       1.101076        2655.298396
USA          2010  479767.392756      268735.997401
             2011  464353.336443      345122.292122
             2012  431056.406089      324876.452673
             2013  392718.657778      284664.563095
             2015  389325.275144      141929.513978
             2016  482808.158989      117573.729269
             2017  528838.972564      162172.964484

In [10]:
process_flows.comtrade_flow?

In [11]:
good_columns=['Qty Unit Code', 'Year', 'Trade Flow', 'Reporter ISO', 'Partner ISO', 'Commodity', 'Qty', 'Trade Value (US$)', 'Reporter_Pt', 'Partner_Pt']

In [47]:
importlib.reload(process_flows)
file_def = [input_file_oil, "Oil", [333]]
in_file = file_def[0]
commodity = file_def[1]
oil_flows = process_flows.comtrade_flow(in_file, commodity, good_columns=good_columns)
oil_flows.initialize(['333'], inB, good_quantity_col = 'Commodity Code', val_fields=['Trade Value (US$)', 'Qty'])

../infrasap/process_flows.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  country_flows['geometry'] = country_flows.apply(lambda x: generate_line_string(x, line_type), axis=1)


In [49]:
oil_flows.save("/home/wb411133/data/Projects/INFRA/FLOWS/OIL_CRUDE_ONLY_2021/")

In [48]:
oil_flows.save_simple_layers("/home/wb411133/data/Projects/INFRA/FLOWS/OIL_CRUDE_ONLY_2021/GEOJSON", "GEOJSON")

../infrasap/process_flows.py:162: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.country_flows['Reporter_Pt'] = self.country_flows['Reporter_Pt'].apply(str)
../infrasap/process_flows.py:163: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.country_flows['Partner_Pt'] = self.country_flows['Partner_Pt'].apply(str)
/home/wb411133/.conda/envs/geog/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pa

In [ ]:
importlib.reload(process_flows)

for file_def in [
                #[input_file_energy, "Energy", []],
                #[input_file_coal, "Coal", []],
                [input_file_oil, "Oil", []],
                #[input_file_gas, "Gas", []]
                ]:
    print(file_def[1])
    in_file = file_def[0]
    commodity = file_def[1]
    coal_flows = process_flows.comtrade_flow(in_file, commodity)
    #coal_flows.initialize(file_def[2], inB)
    #coal_flows.save_simple_layers("/home/wb411133/data/Projects/INFRA/FLOWS/%s/GEOJSON" % commodity, "GEOJSON")

In [ ]:
coal_flows.initialize?

In [ ]:
importlib.reload(process_flows)
base_out_folder = "/home/wb411133/data/Projects/INFRA/FLOWS"
data_res = {}
for file_def in [
                 [input_file_energy, "Energy", [3]],
                 [input_file_coal, "Coal", [8]],
                 #[input_file_gas, "Gas", [8,1]],
                 #[input_file_oil, "Oil", [8]]
                ]:
    print("Processing %s" % file_def[1])
    data_flows = process_flows.comtrade_flow(file_def[0], file_def[1])
    data_flows.initialize(file_def[2], inB)
    select_folder = os.path.join(base_out_folder, file_def[1])
    for file_type in ["SHP"]:
        out_folder = os.path.join(select_folder, file_type)
        data_flows.save(out_folder, file_type)    
    data_res[file_def[1]] = data_flows

# Processing airport data from Heinrich

In [ ]:
airport_flows = "/home/wb411133/data/Projects/INFRA/FLOWS/Airport_Volume.csv"

inD = pd.read_csv(airport_flows)
#inD.drop(['Country Name', 'Country Name.1'], axis=1, inplace=True)

In [ ]:
inD.head()

In [ ]:
# extract airport locations
inD_grouped = inD.groupby(['Orig','Year'])
d = {'Name':'first','TotalSeats':'sum', "Country Name":"first","Airport1Latitude":'first', "Airport1Longitude":'first'}
airport_locations = inD_grouped.agg(d)
airport_locations.head()

In [ ]:
airport_locations = airport_locations.groupby(level=0).last()
airport_locations.head()

In [ ]:
airport_locations.to_csv(airport_flows.replace(".csv", "_airport_locations.csv"))

In [ ]:
#Calculate out_flows
d = {'TotalSeats':'sum'}
inD_grouped = inD.groupby(['Country1', "Year"])
out_flows = inD_grouped.agg(d).reset_index()

#Calculate in_flows
d = {'TotalSeats':'sum'}
inD_grouped = inD.groupby(['Country2', "Year"])
in_flows = inD_grouped.agg(d)

in_flows = in_flows['TotalSeats'].unstack().reset_index()

In [ ]:
def get_data(x):
    x = [y for y in x[2:] if not np.isnan(y)]
    return(x[-1])
in_flows['CURRENT'] = in_flows.apply(get_data, axis=1)
in_flows.shape

# Calculate airport and port flows

In [ ]:
port_data = "/home/public/Data/GLOBAL/INFRA/PORTS/Port_flow_data_Q22020.csv"
port_locations = "/home/public/Data/GLOBAL/INFRA/PORTS/attributed_ports.shp"
airport_data = "/home/wb411133/data/Projects/INFRA/FLOWS/Airport_Volume.csv"

inP = pd.read_csv(port_data)
inP_loc = gpd.read_file(port_locations)
inA = pd.read_csv(airport_data)
# For the simplest version, we are only calculating flows for 2017
inA = inA.loc[inA['Year'] == 2019]

In [ ]:
agg = {'Name':'first','NAme':'first','Country1':'first','Country2':'first','TotalSeats':'sum',"Country Name":"first","Airport1Latitude":'first', "Airport1Longitude":'first',"Airport2Latitude":'first', "Airport2Longitude":'first'}
inA_g = inA.groupby(['Orig','Dest'])
inA_g = inA_g.agg(agg).reset_index()
geoms = inA_g.apply(lambda x: LineString([Point(x['Airport1Longitude'], x['Airport1Latitude']),
                                         Point(x['Airport2Longitude'], x['Airport2Latitude'])]), axis=1)
inA_g = gpd.GeoDataFrame(inA_g, geometry=geoms, crs={'init':'epsg:4326'})


In [ ]:
inA_g.head()

In [ ]:
inA_g.to_file("/home/wb411133/temp/airport_flows.shp")

In [ ]:
# attribute port flows
agg = {'Quarterly deployed capacity (TEU)':'sum'}
inP_g = inP.groupby(["Port1","Port2"])
inP_flows = inP_g.agg(agg).reset_index()

In [ ]:
inP_geoms = inP_flows.merge(inP_loc.loc[:,['LOCODE','geometry']], left_on="Port1", right_on="LOCODE")
inP_geoms.columns = ['Port1','Port2','FLOWS','LOCODE_1','PT_1']
inP_geoms = inP_geoms.merge(inP_loc.loc[:,['LOCODE','geometry']], left_on="Port2", right_on="LOCODE")
inP_geoms.columns = ['Port1','Port2','FLOWS','LOCODE_1','PT_1','LOCODE_2','PT_2']
inP_geoms.drop(['LOCODE_1',"LOCODE_2"], axis=1, inplace=True)
inP_geoms.head()

In [ ]:
sindex = inB.sindex

In [ ]:
inB.loc[list(sindex.nearest([inP_geoms['PT_1'].iloc[0].x, inP_geoms['PT_1'].iloc[0].y]))[0]]['ISO3']

In [ ]:
# attribute port flows with origin and destination country
#inP_geoms['Country1'] = inP_geoms['PT_1'].apply(lambda x: inB.loc[list(sindex.nearest([x.x, x.y]))[0]]['ISO3'])
#inP_geoms['Country2'] = inP_geoms['PT_2'].apply(lambda x: inB.loc[list(sindex.nearest([x.x, x.y]))[0]]['ISO3'])
inP_geoms['Country1'] = inP_geoms['Port1'].apply(lambda x: x[:2])
inP_geoms['Country2'] = inP_geoms['Port2'].apply(lambda x: x[:2])
inP_geoms.head()

In [ ]:
flow_geoms = inP_geoms.apply(lambda x: LineString([x['PT_1'], x['PT_2']]), axis=1)
inP_geoms = gpd.GeoDataFrame(inP_geoms, geometry=flow_geoms, crs={'init':'epsg:4326'})
inP_geoms.drop(["PT_1","PT_2"], axis=1).to_file("/home/wb411133/temp/port_flows.shp")